In [16]:
import torch.nn as nn
from torch.nn import functional as F

In [42]:
learning_rate = 3e-4
max_iters = 50000

In [22]:
import torch

def check_gpu():
    if torch.cuda.is_available():
        device = 'cuda'
        print("GPUs are available.")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        print(f"Device is set to: {device}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("No GPUs available.")
        device = 'cpu'
        print(f"Device is set to: {device}")
    return device

device = check_gpu()


GPUs are available.
Number of GPUs: 1
Device is set to: cuda
GPU 0: NVIDIA GeForce RTX 4070


In [23]:
#M

In [24]:
block_size = 8 
batch_size = 4


In [25]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [26]:
string_to_int = { ch:i for i ,ch in enumerate(chars) }
int_to_string = { i:ch for i ,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [27]:
print(decode(encode('hello')))

hello


In [28]:
data = torch.tensor(encode(text), dtype = torch.long)

In [29]:
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [38]:
n = int(0.8 * len(data))
train_data = data[:n] 
val_data = data[n:] 


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x, y    

In [31]:
block_size = 8 

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = x[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(80)
when input is tensor([80, 28]) target is tensor(28)
when input is tensor([80, 28, 39]) target is tensor(39)
when input is tensor([80, 28, 39, 42]) target is tensor(42)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(49)


In [32]:
x, y = get_batch('train')
x,y = x.to(device) , y.to(device)
print(f"The shape of x is: {x.shape}")
print(f"x is: {x}",  end='\n\n')
print(f"The shape of y is: {y.shape}")
print(f"y is: {y}")

tensor([122419,  67417,  12114,  90497])
The shape of x is: torch.Size([4, 8])
x is: tensor([[58, 72,  9,  1, 54, 67, 57,  1],
        [ 1, 67, 58, 77, 73,  9,  1, 55],
        [58, 62, 73, 61, 58, 71,  1, 73],
        [72,  1, 73, 61, 58,  1, 46, 54]], device='cuda:0')

The shape of y is: torch.Size([4, 8])
y is: tensor([[72,  9,  1, 54, 67, 57,  1, 58],
        [67, 58, 77, 73,  9,  1, 55, 74],
        [62, 73, 61, 58, 71,  1, 73, 61],
        [ 1, 73, 61, 58,  1, 46, 54, 65]], device='cuda:0')


In [33]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim= -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next), dim=1)

        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



3WGSfh1N3DWSA"
t2(jo oRv8MyG*Zpva*9k mUEXNygdC87﻿52]7XjlJER,RzW6P8yocl(x_S'smwXuVSRUo?XXa"n[sKrv3G5J,S*53dKhGl6hTY7(cVsxa&!EB
AhM"w&aE .GF3,OZZ;Nxp020TUmHTjb8X7H-viNP2Ti7XNO;'FXY7vi72OKL3:iR)u("ObR1aGgPHOgNJ,Vwf3Ow&V[k.Y9q2ZA& cu7E*P;fA﻿ I'8u﻿oSBzDr5*WN
I);Z2wKk'sY:jMfr[brP9zaamwXd-GN]M0RD1b*QEYXDrviS1q8v!iM*3O)zG'yds8dmBV2TWR(MbKI
'1
z﻿oaKLi[kbp9ozs
AcDxt EY9]:q1
zT]y!﻿Mx﻿7mlkXhq0fN)ct(xmAjlf8yi35;9IrA:'F&wKx1
fk.6g9O;zHe"&hmtBUmxWZ(1"FNLM(&'Fw1
X[G_dQ
(6McU;NPzt1
gf*XD-G76niiWyWWP;5;U,;dsSdv1!


In [43]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.378742218017578


In [44]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



pey t l izeds thand ks wecaref nt ais k mid he I s acit'sire be ily thio grangos."

T but gnoos  urnde, tr mshe Buthe Oz y  he pung tre higly. acatilins,
S L.

s the thos ce PTh tourewe gruthed ougeve h cerongy, m. boothe ainso; HEutthe  Zed theden  m
tord the upino
hryonellsaluther tamyod, nglan he mbeatw OSum thistind. thato icowe thinthive ikis touly
T fis wereswomud ataruteed  peslde fre mo swh.

" there hengowis o whed. thyo ws brelitisaprra, iry

as cle, seanoman "ong. tis. we, tes asam f
